# This notebook explores the figure 3

In [1]:
%load_ext autoreload
%autoreload 2

from scipy.signal import blackmanharris, gaussian
import os
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import theano
import lasagne
from lasagne.layers import ReshapeLayer,Layer
from lasagne.init import Normal
from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
import convsep.util as util
import numpy as np
import time

from convsep.dataset import LargeDatasetMulti
from trainDNN import build_ca, train_auto
from phase_transform import PhaseTransform


import keras as K


ld2 = LargeDatasetMulti(path_transform_in="results/features", nsources=4,
                    batch_size=2, batch_memory=8,
                    time_context=11, overlap=75, nprocs=4,
                    mult_factor_in=0.3, mult_factor_out=0.3,
                    extra_features=True, model="p")
ld2.extra_feat_size = 1025

load=False
skip_train=False
skip_sep=False
chunk_size=60
chunk_overlap=2
nsamples=40
batch_size=2
batch_memory=8
time_context=30
overlap=25
nprocs=4
mult_factor_in=0.3
mult_factor_out=0.3
db = "../DSD100subset/"
model="dsd_fft_1024"
nepochs = 40
scale_factor = 0.3
tt = PhaseTransform(frameSize=2048, hopSize=512, sampleRate=44100, window=gaussian, std=0.4)

transform = tt
outdir=os.path.join(db,'output',model)
testdir=os.path.join(db,'Mixtures')
model=os.path.join(db,'models',"model_"+model+".pkl")
num_epochs=nepochs
scale_factor=scale_factor



I 2019-03-11 19:37:38 dataset:1127 found 2 files
I 2019-03-11 19:37:38 dataset:1141 iteration size 341


In [2]:

# train_errs = train_auto(train=ld2, fun=build_ca, transform=tt,
#                 outdir=os.path.join(db,'output',model),
#                 testdir=os.path.join(db,'Mixtures'),
#                 model=os.path.join(db,'models',"model_"+model+".pkl"),
#                 num_epochs=nepochs,
#                 scale_factor=scale_factor)

fun = build_ca
train = ld2


sources = ['vocals','bass','drums','other']

nchannels = int(train.channels_in)
nsources = int(train.channels_out/train.channels_in)

print('nchannels: ', nchannels)
print('nsources: ', nsources)

input_size = int(float(transform.frameSize) / 2 + 1)

nchannels:  2
nsources:  4


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.optimizers import Adagrad
import keras.backend as K


def build_model(feat_size=1025, nchannels=2, nsources=4):
    mag_in = Input(shape=(, ))
 = Dense(1, )(mag_in)

#K.bias_add(x, bias, data_format=None)

initial_value = 1
learned = tf.Variable(initial_value, name='learned_scalar')
Lambda(lambda x: x * learned)

second_input = Input(shape=(2, ))
second_dense = Dense(1, )(second_input)

merge_one = concatenate([first_dense, second_dense])

third_input = Input(shape=(1, ))
merge_two = concatenate([merge_one, third_input])

model = Model(inputs=[first_input, second_input, third_input], outputs=merge_two)
model.compile(optimizer=ada_grad, loss='mse',
               metrics=['accuracy'])

In [11]:
def build_ca(amp=None, df_ph=None, dt_ph=None, batch_size=8, time_context=5, feat_size=1025, nchannels=2, nsources=4):

    input_shape=(batch_size, nchannels, time_context, feat_size)
    n_hiddens = 500
    out_shape = (batch_size, nchannels*nsources, feat_size)
    print(input_shape)
    print(out_shape)
    # input layer for amplitude features
    a_in = lasagne.layers.FlattenLayer(lasagne.layers.InputLayer(shape=input_shape, input_var=amp))
    b_in = lasagne.layers.FlattenLayer(lasagne.layers.InputLayer(shape=input_shape, input_var=df_ph))
    c_in = lasagne.layers.FlattenLayer(lasagne.layers.InputLayer(shape=input_shape, input_var=dt_ph))
    
    d = lasagne.layers.ConcatLayer([a_in, b_in, c_in])
    
    # DEBUG
    d = lasagne.layers.DenseLayer(d, nchannels*nsources*feat_size,
                                    nonlinearity=lasagne.nonlinearities.rectify)
    d_out = lasagne.layers.ReshapeLayer(d, (out_shape))
    return d_out

network = build_ca(amp=amp, df_ph=df_ph, dt_ph=dt_ph,
        batch_size=2,time_context=5,
        feat_size=1025, nchannels=nchannels, nsources=nsources)

prediction = lasagne.layers.get_output(network, deterministic=True)

sourceall=[]
errors_insts = []
loss = 0

sep_chann = []

loss = lasagne.objectives.squared_error(prediction, abs(target_var[:,:,5,:]))
loss = loss.mean()

(2, 2, 5, 1025)
(2, 8, 1025)


TypeError: Join() can only join tensors with the same number of dimensions.

In [ ]:
import time 

params1 = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.adadelta(loss, params1)

losser=[]

train_fn = theano.function([amp, df_ph, dt_ph, target_var], loss, updates=updates, allow_input_downcast=True)

In [8]:
df_ph.dtype

dtype('float64')

In [10]:
theano.config.exception_verbosity = "high"

if not skip_train:
#     for epoch in range(num_epochs):
#         train_err = 0
#         train_batches = 0
#         errs = np.zeros((nchannels,nsources))
#         start_time = time.time()
#         for batch in range(train.iteration_size):
#             mag, target, features = train()
#             df_ph, dt_ph = features[..., :]
#             train_err += train_fn_mse(mag, target)
#             errs += np.array(train_fn1(mag, df_ph, dt_ph, target))
#             train_batches += 1

#         logging.info("Epoch {} of {} took {:.3f}s".format(
#             epoch + 1, num_epochs, time.time() - start_time))
#         logging.info("  training loss:\t\t{:.6f}".format(train_err/train_batches))
#         for j in range(nchannels):
#             for i in range(nsources):
#                 logging.info("  training loss for "+sources[i]+" in mic "+str(j)+":\t\t{:.6f}".format(errs[j][i]/train_batches))

#         model_noILD = model[:-4] + '_noILD' + model[-4:]
#         print('model_noILD: ', model_noILD)
#         save_model(model_noILD,network)
#         losser.append(train_err/train_batches)
        
        
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        errs = np.zeros((nchannels,nsources))
        for batch in range(train.iteration_size):
            mag, target, features = train()
            df_ph = features[..., 0]
            dt_ph = features[..., 1]
            print(df_ph.shape, dt_ph.shape, mag.shape, target.shape)
            errs += np.array(train_fn(mag, df_ph, dt_ph, target))
            train_batches += 1


        # And a full pass over the validation data:
#         val_err = 0
#         val_acc = 0
#         val_batches = 0
#         for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
#             inputs, targets = batch
#             err, acc = val_fn(inputs, targets)
#             val_err += err
#             val_acc += acc
#             val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(err / train_batches))
#         print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
#         print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
#         print("  validation accuracy:\t\t{:.2f} %".format(
#             val_acc / val_batches * 100))

(2, 2, 11, 1025) (2, 2, 11, 1025) (2, 2, 11, 1025) (2, 8, 11, 1025)


ValueError: Shape mismatch: x has 67650 cols (and 2 rows) but y has 30750 rows (and 8200 cols)
Apply node that caused the error: Dot22(Join.0, W)
Toposort index: 16
Inputs types: [TensorType(float32, matrix), TensorType(float32, matrix)]
Inputs shapes: [(2, 67650), (30750, 8200)]
Inputs strides: [(270600, 4), (32800, 4)]
Inputs values: ['not shown', 'not shown']
Inputs type_num: [11, 11]
Outputs clients: [[Elemwise{Add}[(0, 0)](Dot22.0, InplaceDimShuffle{x,0}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/pierre-louis/.pyenv/versions/3.5.6/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2869, in _run_cell
    return runner(coro)
  File "/home/pierre-louis/.pyenv/versions/3.5.6/lib/python3.5/site-packages/IPython/core/async_helpers.py", line 67, in _pseudo_sync_runner
    coro.send(None)
  File "/home/pierre-louis/.pyenv/versions/3.5.6/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 304

In [ ]:
inputs, targets, features = train()

In [ ]:
train_fn(mag, df_ph, dt_ph, target)